# Colab-RN
Original repo: [geekyutao/RN](https://github.com/geekyutao/RN)

Differentiable Augmentation: [mit-han-lab/data-efficient-gans](https://github.com/mit-han-lab/data-efficient-gans)

My fork: [styler00dollar/Colab-RN](https://github.com/styler00dollar/Colab-RN)

In [ ]:
!nvidia-smi

In [ ]:
#@title git clone and install
!git clone https://github.com/styler00dollar/Colab-RN
!pip install scipy==1.1
!pip install tensorboardX
!pip install scikit-image
!pip install opencv-python
!pip install torchvision

Training

In [ ]:
#@title Create empty folders
#!mkdir /content/training-checkpoints/
!mkdir /content/train/
#!mkdir /content/val/
!mkdir /content/mask_train/
#!mkdir /content/mask_val/

In [ ]:
#@title training
%cd /content/Colab-RN
!python main.py --bs 1 --gpus 1 --prefix rn \
--img_flist /content/train/train.tflist \
--mask_flist /content/mask_train/mask_train.tflist --bs 1 --amp 1 --print_interval 1

Testing


Sidenote: Uses black to mark areas instead of white.

In [ ]:
#@title Image and mask need to be dividable by 4, this code does fix wrong images 
import cv2
import numpy
path_inpainting = '/content/val/0.jpg'
path_mask = '/content/mask_val/0.png'
image=cv2.imread(path_mask)
image_size0 = numpy.floor(image.shape[0]/4)
image_size1 = numpy.floor(image.shape[1]/4)
image=cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
ret,image=cv2.threshold(image,254,255,cv2.THRESH_BINARY)
image = cv2.resize(image, (int(image_size1*4), int(image_size0*4)), cv2.INTER_NEAREST)
cv2.imwrite(path_mask, image)

image=cv2.imread(path_inpainting)
image = cv2.resize(image, (int(image_size1*4), int(image_size0*4)), cv2.INTER_NEAREST)
cv2.imwrite(path_inpainting, image)

!convert /content/mask_val/0.png -channel RGB -negate /content/mask_val/0.png

In [ ]:
!python eval.py --bs 1 --model /content/RN/pretrained_model/x_admin.cluster.localRN-0.8RN-Net_bs_14_epoch_3.pth \
--img_flist /content/val/val.tflist --mask_flist /content/mask_val/mask_val.tflist 

In [ ]:
#@title removing alpha from image (Example: val/0.jpg)
import cv2
filename = '/content/val/0.jpg'
image = cv2.imread(filename)
cv2.imwrite(filename, image)